In [1]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.test.is_gpu_available())

Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is available
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0OLBYI/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0OLBYI/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0OLBYI/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Training entries: 25000, test entries: 25000


In [3]:
train_examples[:2], train_labels[:2]

(array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
        b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plo

In [4]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

In [5]:
def load_data(examples, targets, num_words, sequence_length, do_test = False, test_size=0.20, oov_token=None):

    reviews, labels = [], []

    for example, label in zip(examples, targets):
      reviews.append(str(example).strip())
      labels.append(str(label).strip())

    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    tokenizer.fit_on_texts(reviews)
    X = tokenizer.texts_to_sequences(reviews)
    X, y = np.array(X), np.array(labels)
    X = pad_sequences(X, maxlen=sequence_length)

    # convert labels to one-hot encoded
    y = to_categorical(y)

    
    if do_test == False:
      print ("begin loding training data...")
      X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=1)
      data = {}
      data["X_train"] = X_train
      data["X_val"] = X_val
      data["y_train"] = y_train
      data["y_val"] = y_val
      data["tokenizer"] = tokenizer
      data["int2label"] =  {0: "negative", 1: "positive"}
      data["label2int"] = {"negative": 0, "positive": 1}

      return data
    else:
      print ("begin loding test data...")
      data = {}
      data["X_test"] = X
      data["y_test"] = y
      data["tokenizer"] = tokenizer
      data["int2label"] =  {0: "negative", 1: "positive"}
      data["label2int"] = {"negative": 0, "positive": 1}

    return data


In [6]:
class Classifier:

  def __init__(self, word_index, embedding_dims, max_len, num_class = 2):

    self.vocab_size = len(word_index) + 1
    self.embedding_dims = embedding_dims
    self.max_len = max_len
    self.num_class = num_class

  def create_model(self):
    model = Sequential([
        tf.keras.layers.Embedding(self.vocab_size, self.embedding_dims, input_length = self.max_len),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(self.num_class),
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    return model

In [7]:
embedding_dims = 300
max_len= 100

data = load_data(train_examples, train_labels, 10000, 100, do_test = False)

classifier = Classifier(data["tokenizer"].word_index, embedding_dims, max_len)

begin loding training data...


In [8]:
model = classifier.create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          25961700  
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 25,981,094
Trainable params: 25,981,094
Non-trainable params: 0
_________________________________________________________________


In [9]:
if not os.path.isdir("logs"):
    os.mkdir("logs")

tensorboard = TensorBoard(log_dir=os.path.join("logs", "IMDB"))

history = model.fit(data["X_train"], data["y_train"],
                    batch_size=256,
                    epochs=10,
                    validation_data=(data["X_val"], data["y_val"]),
                    callbacks=[tensorboard],
                    verbose=1)

model.save("IMDB.h5", overwrite=True, include_optimizer = True, save_format="tf")

Epoch 1/10
79/79 [==============================] - 23s 294ms/step - loss: 0.6920 - accuracy: 0.6324 - val_loss: 0.6897 - val_accuracy: 0.7064
Epoch 2/10
79/79 [==============================] - 22s 284ms/step - loss: 0.6852 - accuracy: 0.7399 - val_loss: 0.6789 - val_accuracy: 0.7378
Epoch 3/10
79/79 [==============================] - 23s 285ms/step - loss: 0.6676 - accuracy: 0.7638 - val_loss: 0.6548 - val_accuracy: 0.7600
Epoch 4/10
79/79 [==============================] - 22s 283ms/step - loss: 0.6341 - accuracy: 0.7859 - val_loss: 0.6157 - val_accuracy: 0.7814
Epoch 5/10
79/79 [==============================] - 22s 284ms/step - loss: 0.5863 - accuracy: 0.8054 - val_loss: 0.5680 - val_accuracy: 0.7980
Epoch 6/10
79/79 [==============================] - 23s 288ms/step - loss: 0.5320 - accuracy: 0.8227 - val_loss: 0.5193 - val_accuracy: 0.8114
Epoch 7/10
79/79 [==============================] - 22s 283ms/step - loss: 0.4797 - accuracy: 0.8370 - val_loss: 0.4767 - val_accuracy: 0.8172

In [10]:
new_model = tf.keras.models.load_model("IMDB.h5")
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          25961700  
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                19264     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 25,981,094
Trainable params: 25,981,094
Non-trainable params: 0
_________________________________________________________________


In [12]:
text = "The movie is awesome!"
sequence = data["tokenizer"].texts_to_sequences([text])
# pad the sequences
sequence = pad_sequences(sequence, maxlen=max_len)
prediction = new_model.predict(sequence)[0]
print(prediction)
print(data["int2label"][np.argmax(prediction)])

[-0.6756521   0.69552356]
positive
